In [1]:
from google.colab import files
files.upload()

!mkdir -p ~/.kaggle//
!cp kaggle.json ~/.kaggle/
# Permission Warning이 발생하지 않도록 해줍니다.
!chmod 600 ~/.kaggle/kaggle.json

Saving kaggle.json to kaggle.json


In [28]:
import time
start_time = time.time()
from sklearn.model_selection import train_test_split
import sys, os, re, csv, codecs, numpy as np, pandas as pd
np.random.seed(32)
os.environ['OMP_NUM_THREADS'] = '8'

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation, Conv1D, GRU
from keras.layers import Bidirectional, GlobalMaxPool1D, MaxPooling1D, Add, Flatten
from keras.layers import GlobalAveragePooling1D, GlobalMaxPooling1D, concatenate, SpatialDropout1D
from keras.models import Model, load_model
from keras import initializers, regularizers, constraints, optimizers, layers, callbacks
from keras import backend as K
from keras.layers import InputSpec, Layer

import logging
from sklearn.metrics import roc_auc_score
from keras.callbacks import Callback

In [3]:
class RocAucEvaluation(Callback):
  def __init__(self, validation_data=(), interval=1):
    super(Callback, self).__init__()

    self.interval = interval
    self.X_val, self.y_val = validation_data

  def on_epoch_end(self, epoch, logs={}):
    if epoch % self.interval == 0:
      y_pred = self.model.predict(self.X_val, verbose=0)
      score = roc_auc_score(self.y_val, y_pred)
      print(f'\n ROC-AUC - epoch: {epoch+1} - score: {score:.6f}\n')

In [4]:
NLP_ENG_DIR = '/content/drive/MyDrive/NLP/ENG/'
WV_DIR = NLP_ENG_DIR + 'wordvector/'
BASE_DIR = NLP_ENG_DIR + 'Jigsaw1/'
DATA_DIR = BASE_DIR + 'input/'
OUTPUT_DIR = BASE_DIR + 'output/'
MODEL_DIR = BASE_DIR + 'model/'

In [11]:
train = pd.read_csv(DATA_DIR + 'train.csv')
test = pd.read_csv(DATA_DIR + 'test.csv')

embedding_path = WV_DIR + 'glove/glove.840B.300d.txt'
embed_size = 300 # size of vector for each word
max_feature = 130000 # number of words to use as feature
max_len = 220 # sequence len

In [12]:
list_classes = ["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]
y = train[list_classes].values
train["comment_text"].fillna("no comment")
test["comment_text"].fillna("no comment")
X_train, X_valid, Y_train, Y_valid = train_test_split(train, y, test_size = 0.1)

In [13]:
raw_text_train = X_train["comment_text"].str.lower()
raw_text_valid = X_valid["comment_text"].str.lower()
raw_text_test = test["comment_text"].str.lower()

In [14]:
tk = Tokenizer(num_words = max_feature, lower = True)
tk.fit_on_texts(raw_text_train)
X_train["comment_seq"] = tk.texts_to_sequences(raw_text_train)
X_valid["comment_seq"] = tk.texts_to_sequences(raw_text_valid)
test["comment_seq"] = tk.texts_to_sequences(raw_text_test)

In [15]:
X_train = pad_sequences(X_train.comment_seq, maxlen = max_len)
X_valid = pad_sequences(X_valid.comment_seq, maxlen = max_len)
test = pad_sequences(test.comment_seq, maxlen = max_len)

In [16]:
def get_coefs(word,*arr): return word, np.asarray(arr, dtype='float32')
embedding_index = dict(get_coefs(*o.strip().split(" ")) for o in open(embedding_path))

In [18]:
word_index = tk.word_index
nb_words = min(max_feature, len(word_index))
embedding_matrix = np.zeros((nb_words, embed_size))
for word, i in word_index.items():
    if i >= max_feature: continue
    embedding_vector = embedding_index.get(word)
    if embedding_vector is not None: embedding_matrix[i] = embedding_vector

In [33]:
from keras.optimizers import adam_v2, rmsprop_v2
from keras.callbacks import EarlyStopping, ModelCheckpoint, LearningRateScheduler
from keras.layers import GRU, BatchNormalization, Conv1D, MaxPooling1D

file_path = MODEL_DIR + "bigru_cnn_pooling2.hdf5"
check_point = ModelCheckpoint(file_path, monitor = "val_loss", verbose = 1,
                              save_best_only = True, mode = "min")
ra_val = RocAucEvaluation(validation_data=(X_valid, Y_valid), interval = 1)
early_stop = EarlyStopping(monitor = "val_loss", mode = "min", patience = 5)

In [34]:
def build_model(lr = 0.0, lr_d = 0.0, units = 0, dr = 0.0):
    inp = Input(shape = (max_len,))
    x = Embedding(max_feature, embed_size, weights = [embedding_matrix], trainable = False)(inp)
    x1 = SpatialDropout1D(dr)(x)

    ## GRU X CNN
    x = Bidirectional(GRU(units, return_sequences = True))(x1)
    x = Conv1D(64, kernel_size = 2, padding = "valid", kernel_initializer = "he_uniform")(x)
    
    ## LSTM X CNN
    y = Bidirectional(LSTM(units, return_sequences = True))(x1)
    y = Conv1D(64, kernel_size = 2, padding = "valid", kernel_initializer = "he_uniform")(y)
    
    avg_pool1 = GlobalAveragePooling1D()(x)
    max_pool1 = GlobalMaxPooling1D()(x)
    
    avg_pool2 = GlobalAveragePooling1D()(y)
    max_pool2 = GlobalMaxPooling1D()(y)
    
    x = concatenate([avg_pool1, max_pool1, avg_pool2, max_pool2])

    x = Dense(6, activation = "sigmoid")(x)
    model = Model(inputs = inp, outputs = x)
    model.compile(loss = "binary_crossentropy", optimizer = adam_v2.Adam(lr = lr, decay = lr_d), metrics = ["accuracy"])
    history = model.fit(X_train, Y_train, batch_size = 128, epochs = 3, validation_data = (X_valid, Y_valid), 
                        verbose = 1, callbacks = [ra_val, check_point, early_stop])
    model = load_model(file_path)
    return model

In [35]:
model = build_model(lr = 1e-3, lr_d = 0, units = 128, dr = 0.2)
pred = model.predict(test, batch_size = 1024, verbose = 1)

Epoch 1/3


/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


1122/1122 [==============================] - ETA: 0s - loss: 0.0529 - accuracy: 0.8897
 ROC-AUC - epoch: 1 - score: 0.987369


Epoch 1: val_loss improved from inf to 0.04485, saving model to /content/drive/MyDrive/NLP/ENG/Jigsaw1/model/bigru_cnn_pooling2.hdf5
1122/1122 [==============================] - 122s 103ms/step - loss: 0.0529 - accuracy: 0.8897 - val_loss: 0.0448 - val_accuracy: 0.9582
Epoch 2/3
1122/1122 [==============================] - ETA: 0s - loss: 0.0418 - accuracy: 0.9211
 ROC-AUC - epoch: 2 - score: 0.988282


Epoch 2: val_loss improved from 0.04485 to 0.04282, saving model to /content/drive/MyDrive/NLP/ENG/Jigsaw1/model/bigru_cnn_pooling2.hdf5
1122/1122 [==============================] - 113s 101ms/step - loss: 0.0418 - accuracy: 0.9211 - val_loss: 0.0428 - val_accuracy: 0.9021
Epoch 3/3
1122/1122 [==============================] - ETA: 0s - loss: 0.0387 - accuracy: 0.9201
 ROC-AUC - epoch: 3 - score: 0.988887


Epoch 3: val_loss improved from 0.04282 to 0.04074, sav

In [36]:
submission = pd.read_csv(DATA_DIR + "sample_submission.csv")
submission[list_classes] = (pred)
submission.to_csv(OUTPUT_DIR+"bigru_cnn_pooling2_adamv2.csv", index = False)

In [24]:
submission.head() # tf.keras.Adam

,id,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,00001cee341fdb12,0.993993,0.568490,0.987547,0.146149,0.886568,0.484644
1,0000247867823ef7,0.000975,0.000033,0.000492,0.000015,0.000241,0.000094
2,00013b17ad220c46,0.000676,0.000297,0.000394,0.000047,0.000219,0.000199
3,00017563c3f7919a,0.000314,0.000014,0.000338,0.000052,0.000289,0.000059
4,00017695ad8997eb,0.009652,0.000433,0.002324,0.000236,0.000670,0.000249


In [37]:
submission.head() # adamv2

,id,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,00001cee341fdb12,0.998601,0.603103,0.976610,0.199668,0.900500,0.482347
1,0000247867823ef7,0.000994,0.000018,0.000383,0.000020,0.000636,0.000053
2,00013b17ad220c46,0.001522,0.000106,0.000667,0.000089,0.000761,0.000101
3,00017563c3f7919a,0.000391,0.000006,0.000160,0.000062,0.000323,0.000027
4,00017695ad8997eb,0.021458,0.000314,0.001039,0.000465,0.000899,0.000065


In [38]:
!kaggle competitions submit -c jigsaw-toxic-comment-classification-challenge -f /content/drive/MyDrive/NLP/ENG/Jigsaw1/output/bigru_cnn_pooling2_adamv2.csv -m "bigru_pooling_cnn2_adamv2"

100% 13.7M/13.7M [00:01<00:00, 7.35MB/s]
Successfully submitted to Toxic Comment Classification Challenge